### Logistic regression

- Implement a logistic regression model using a neural network mindset.
- Code includes computing forward and backward propagation, gradients, and updating model parameters.

*Pseudocode of the algorithm*:

Forward step:
$$Z = w^{T}X + b$$
$$A = \sigma(Z)$$

Backward step:
$$dZ = A - Y$$
$$dw = \frac{1}{m} XdZ^{T}$$
$$db = \frac{1}{m} \sum^{m}_{i=1}{dZ^{(i)}}$$

Cost function:
$$J(w,b) = -\frac{1}{m}\sum_{i=1}^{m}(y^{(i)}log(a^{(i)}) + (1-y^{(i)})log(1-a^{(i)}))$$
Update weights:
$$w := w - \alpha dw$$
$$b := b - \alpha db$$

In [41]:
import numpy as np
import h5py
from lr_utils import load_dataset

In [42]:
def initialize(nx):
    """Initialize the parameters of the logistic regression model"""
    w = np.zeros((nx, 1))
    b = 0
    return w, b

In [43]:
def sigmoid(Z):
    """Compute the sigmoid function"""
    return 1 / (1+np.exp(-Z))

In [44]:
def forward(w, X, b):
    """Forward propagation"""
    Z = np.dot(w.T, X) + b
    A = sigmoid(Z)
    return A

In [45]:
def backward(A, Y, X, m):
    """Backward propagation"""
    dZ = A - Y
    dw = np.dot(X, dZ.T) / m
    db = np.sum(dZ) / m
    return dw, db


In [46]:
def cost(A, Y, m):
    """Calculate the cost function across training examples"""
    J = -1/m * (np.dot(Y, np.log(A).T) + np.dot(1-Y, np.log(1-A).T))
    J = np.squeeze(J)
    J = np.round(J,5)
    return J

In [47]:
def update_param(w, b, dw, db, alpha):
    """Update parameters w and b based on the derivatives computed using gradient descent"""
    w = w - alpha * dw
    b = b - alpha * db
    return w, b

In [48]:
def load_data():
    """
    Load the data, standardize, and reshape images into an nx by m feature matrix
    """
    trainX, trainY, testX, testY, classes = load_dataset()
    trainX = trainX / 255
    testX = testX / 255
    trainX = trainX.reshape((trainX.shape[0],-1)).T
    testX = testX.reshape((testX.shape[0],-1)).T
    return trainX, trainY, testX, testY, classes

In [49]:
def model(X, Y, nx, m, nepoch=2000, alpha=0.005, print_cost=True):
    """
    Optimize w and b by running gradient descent
    (1) Initialize weights
    (2) Forward propagation
    (3) Compute cost J
    (4) Backward propagation
    (5) Update weight and intercept params
    """
    w, b = initialize(nx)
    for i in range(1, nepoch+1):
        A = forward(w, X, b)
        J = cost(A, Y, m)
        if print_cost==True and i%250==0:
            print('Step ', i, ', cost=', J, sep='')
        dw, db = backward(A, Y, X, m)
        w, b = update_param(w, b, dw, db, alpha)
    print()
    return w, b

In [50]:
def predict(X, w, b):
    """Predict on test data using the trained model"""
    A = forward(w, X, b)
    A = A>=0.5
    A = A.astype(int)
    return A

In [51]:
def main():
    trainX, trainY, testX, testY, classes = load_data()
    nx = trainX.shape[0]
    m = trainX.shape[1]
    print('Shape of the training data (nx, m): ', nx, m)
    w, b = model(trainX, trainY, nx, m)
    predY = predict(testX, w, b)
    print('Test accuracy: ', round(np.sum(predY==testY)/testX.shape[1] * 100,2), '%', sep='')
    predY = predict(trainX, w, b)
    print('Training accuracy: ', round(np.sum(predY==trainY)/trainX.shape[1] * 100,2), '%', sep='')

In [52]:
if __name__ == '__main__':
    main()

Shape of the training data (nx, m):  12288 209
Step 250, cost=0.425
Step 500, cost=0.30353
Step 750, cost=0.25136
Step 1000, cost=0.21494
Step 1250, cost=0.18777
Step 1500, cost=0.1666
Step 1750, cost=0.14961
Step 2000, cost=0.13566

Test accuracy: 70.0%
Training accuracy: 99.04%
